# Creating Config Files

This notebook generates config files to specify supernova specific priors 
and fitting arguments. Individual files are created for different supernova 
surveys. The config files are save in yaml format and values typically follow the 
following template:

    salt2:
      object_id:
        kwargs:
          global:
            bounds:
              t0:
                - ? # Typically t0 - 10
                - ? # Typically t0 + 10
            phase_range:
              - -20
              - 50
        priors:
          global:
            z: ?
            t0: ?

    sn91bg:
      object_id:
        kwargs:
          global:
            bounds:
              c:
                - 0
                - 1
              x1:
                - 0.65
                - 1.25
              t0:
                - ? # Typically t0 - 10
                - ? # Typically t0 + 10
            phase_range:
              - -20
              - 50
        priors:
          global:
            z: ?
            t0: ?

We note that values for some supernovae may have been manually  altered in the
config files after manually inspecting their  light curve fits.

In [ ]:
import sys

import numpy as np
from sndata.sdss import sako18
from sndata.csp import dr1
from sndata.des import sn3yr
import yaml

from pathlib import Path

out_dir = Path(some_directory_name)
out_dir.mkdir(exist_ok=True, parents=True)


In [ ]:
def create_config_dict(id_arr, t0_arr, z_arr, t0_range=10):
    
    config_dict = {'salt2': {}, 'sn91bg': {}}
    for obj_id, t0, z in zip(id_arr, t0_arr, z_arr):

        config_dict['salt2'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['salt2'][obj_id]['priors'] = {'global': {'z': z, 't0': t0}}
        config_dict['salt2'][obj_id]['kwargs'] = {'global': {'bounds': {}, 'phase_range': {}}}
        config_dict['salt2'][obj_id]['kwargs']['global']['bounds'] = {'t0': [t0 - 10, t0 + 10]}
        config_dict['salt2'][obj_id]['kwargs']['global']['phase_range'] = [-20, 50]

        config_dict['sn91bg'][obj_id] = {'priors': {}, 'kwargs': {}}
        config_dict['sn91bg'][obj_id]['priors'] = {'global': {'z': z, 't0': t0}}
        config_dict['sn91bg'][obj_id]['kwargs'] = {'global': {'bounds': {}, 'phase_range': {}}}
        config_dict['sn91bg'][obj_id]['kwargs']['global']['bounds'] = {'t0': [t0 - 10, t0 + 10], 'c': [0, 1], 'x1': [0.65, 1.25]}
        config_dict['sn91bg'][obj_id]['kwargs']['global']['phase_range'] = [-20, 50]
    
    return config_dict

def raise_path_exists(path):
    if path.exists():
        raise ValueError(
            'Existing files may have manually modified values.'
            ' Be careful not to overwrite them!'
        )
    

## SDSS - Sako et al. 2018

In [ ]:
master = sako18.load_table('master').to_pandas()
master['JD'] = master['MJDatPeakrmag'] + 2400000.5
master.head()


In [ ]:
classifications = set(master['Classification'])
classifications


In [ ]:
targets_with_redshift = master[master['zCMB'] >= 0]

skip_types = ('AGN', 'Variable', 'SNII', 'zSNII', 'pSNII')
indices = ~np.isin(targets_with_redshift['Classification'], skip_types)
targets_of_interest = targets_with_redshift[indices]

print('Total targets of interest:', len(targets_of_interest), '/', len(master))


In [ ]:
sdss_config = create_config_dict(
    targets_of_interest['CID'],
    targets_of_interest['JD'], 
    targets_of_interest['zCMB'])


In [ ]:
sdss_out_path = out_dir / Path('sdss_config.yml')
raise_path_exists(sdss_out_path)
with open(sdss_out_path, 'w') as ofile:
    yaml.dump(sdss_config, ofile)


## CSP - DR1

In [ ]:
csp_table_1 = dr1.load_table(1).to_pandas()
csp_table_1['JD'] = csp_table_1['T0'] + 2400000.5
csp_table_1.head()


In [ ]:
csp_config = create_config_dict(
    csp_table_1['SN'],
    csp_table_1['JD'], 
    csp_table_1['zCMB'])



In [ ]:
csp_out_path = out_dir / Path('csp_config.yml')
raise_path_exists(csp_out_path)
with open(csp_out_path, 'w') as ofile:
    yaml.dump(csp_config, ofile)


## DES - SN3yr

In [ ]:
des_fit_res = sn3yr.load_table('SALT2mu_DES+LOWZ_C11.FITRES').to_pandas()
des_fit_res['JD'] = des_fit_res['PKMJD'] + 2400000.5
des_fit_res.CIDint = des_fit_res.CIDint.apply(lambda x: str(x).zfill(8))
des_fit_res.head()


In [ ]:
des_config = create_config_dict(
    des_fit_res['CIDint'],
    des_fit_res['JD'], 
    des_fit_res['zCMB'])


In [ ]:
des_out_path = out_dir / Path('des_config.yml')
raise_path_exists(des_out_path)
with open(des_out_path, 'w') as ofile:
    yaml.dump(des_config, ofile)
    